1. Install Gradio

In [9]:
!pip install gradio transformers


In [10]:
!pip install --upgrade fsspec datasets


  Using cached fsspec-2025.3.2-py3-none-any.whl.metadata (11 kB)


In [11]:
! pip install datasets

2. Load the model & tokenizer

In [12]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

model_path = "/content/drive/MyDrive/emotion_model"  # this is roberta
tokenizer  = AutoTokenizer.from_pretrained(model_path)
model      = AutoModelForSequenceClassification.from_pretrained(model_path)

classifier = pipeline(
    "text-classification",
    model=model,
    tokenizer=tokenizer,
    function_to_apply="sigmoid",  # independent multi-label scores
    top_k=3                        # show top 3 emotions
)


Device set to use cpu


3. Define a prediction function

In [14]:
from datasets import load_dataset
go = load_dataset("go_emotions", revision="main")
emotion_names = go["train"].features["labels"].feature.names
print(emotion_names)


['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral']


In [15]:
def predict_emotions(text):
    raw = classifier(text)[0]  # e.g. [{'label':'LABEL_14','score':0.82}, …]
    # map LABEL_i → emotion_names[i]
    return { emotion_names[int(x["label"].split("_")[1])] : float(x["score"])
             for x in raw }


4. Build the Gradio interface

In [16]:
import gradio as gr

iface = gr.Interface(
    fn=predict_emotions,
    inputs=gr.Textbox(lines=4, placeholder="Type your review here…"), # Change to gr.Textbox
    outputs=gr.Label(num_top_classes=3), # Change to gr.Label
    title="Emotion Classifier Demo",
    description="Paste in any text and see the top 3 predicted emotions (with scores)."
)

iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://35e72ec68b95ebd1c8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
